In [10]:
from SoundStream import soundstream_16khz, audio_to_tokens, tokens_to_audio
import torchaudio
import torch
from hubertKM import SemanticTokenizer, visualizeEmbeddings

In [14]:
model = soundstream_16khz()

w2vBERT = SemanticTokenizer("facebook/hubert-base-ls960","./hubertKM/hubert_base_ls960_L9_km500.bin")  

audio_paths = {f'./data_cut/14/342/canterburytales_09_chaucer_64kb_000{i}.flac' for i in range(6)}

#audio_paths.update({f'./data_cut/16/342/canterburytales_24_chaucer_64kb_000{i}.flac' for i in range(10)})
#audio_paths.update({f'./data_cut/16/352/little_lame_prince_01_64kb_000{i}.flac' for i in range(10)})
#audio_paths.update({f'./data_cut/16/352/little_lame_prince_01_64kb_001{i}.flac' for i in range(5)})
#audio_paths.update({f'./data_cut/16/352/little_lame_prince_02_64kb_000{i}.flac' for i in range(10)})
#audio_paths.update({f'./data_cut/16/352/little_lame_prince_02_64kb_001{i}.flac' for i in range(9)})
#audio_paths.update({f'./data_cut/16/352/little_lame_prince_03_64kb_000{i}.flac' for i in range(7)})




start = 0
duration = 30

previouse_coarse_shape = None
previouse_fine_shape = None
previouse_semantic_shape = None
equals = True
for audio_path in audio_paths:
	waveform, sample_rate = torchaudio.load(audio_path)

	x, sample_rate = torchaudio.functional.resample(waveform, sample_rate, 16000), 16000

	waveform = x[:, start*16000:(start + duration)*16000]

	num_samples = waveform.size(1)  # waveform is a tensor of shape (num_channels, num_samples)
	audio_duration = num_samples / sample_rate
	with torch.no_grad():
		semanticTokens, _ = w2vBERT(waveform)
	coarse, fine = audio_to_tokens(waveform, sample_rate, model, start, duration)
	if previouse_coarse_shape == None:
		previouse_coarse_shape = coarse.shape[0]
		previouse_fine_shape = fine.shape[0]
		previouse_semantic_shape = semanticTokens.shape[0]
	else:
		if previouse_coarse_shape != coarse.shape[0] or previouse_fine_shape != fine.shape[0] or previouse_semantic_shape != semanticTokens.shape[0]:
			print("Prevoius: " + str(previouse_coarse_shape) + "  " + str(previouse_fine_shape) + "\nCurrent: " + str(coarse.shape[0]) + " " + str(fine.shape[0]))
			print(audio_path)
			if(audio_duration < start + duration):
				print("Audio duration lesser than cut")
				continue
			equals = False
			break

if equals:
	print("Most of the tokens have an equal measure of:\nCoarse: " + str(previouse_coarse_shape) + "\nFine: " + str(previouse_fine_shape) + "\nSemantic: " + str(previouse_semantic_shape))
else:
	print("The tokens have a different size")

#audio = tokens_to_audio(coarse, fine, model)

Some weights of the model checkpoint at facebook/hubert-base-ls960 were not used when initializing HubertModel: ['encoder.pos_conv_embed.conv.weight_g', 'encoder.pos_conv_embed.conv.weight_v']
- This IS expected if you are initializing HubertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing HubertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of HubertModel were not initialized from the model checkpoint at facebook/hubert-base-ls960 and are newly initialized: ['encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'encoder.pos_conv_embed.conv.parametrizations.weight.original1']
You should probably TRAIN this model on a down-stream task to be able to use it for pre

Most of the tokens have an equal measure of:
Coarse: 4503
Fine: 7505
Semantic: 1499
